In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [3]:
from pyspark.ml.recommendation import ALS

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [6]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [8]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [10]:
train_data,test_data = data.randomSplit([0.8,0.2])

In [11]:
als = ALS(maxIter=5,regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [12]:
model = als.fit(train_data)

In [13]:
predictions = model.transform(test_data)

In [14]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    13| 0.21251455|
|     65|   1.0|    28|-0.88462925|
|     65|   1.0|    22|  1.8452443|
|     65|   2.0|    15| 0.84935915|
|     65|   5.0|    23|-0.52882975|
|     53|   3.0|    13|-0.17921829|
|     78|   1.0|    12| 0.13174367|
|     78|   1.0|    22|-0.31898916|
|     78|   1.0|    13|  1.2535743|
|     78|   1.0|    20| 0.09121838|
|     78|   1.0|    19|  0.6586407|
|     78|   1.0|    17| 0.28234255|
|     78|   1.0|     4|  0.8662399|
|     78|   1.0|    24| 0.71321565|
|     78|   1.0|    11|  1.0908246|
|     34|   1.0|    28| 0.24819665|
|     34|   1.0|    15|  1.4469912|
|     34|   1.0|     4|  1.6780291|
|     81|   1.0|    15|  1.9882041|
|     81|   1.0|     7| 0.52945364|
+-------+------+------+-----------+
only showing top 20 rows



In [18]:
evaluator = RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')

In [19]:
rmse = evaluator.evaluate(predictions)

In [20]:
print(rmse)

2.210600277798214


In [27]:
single =test_data.filter(test_data['userId']==17).select(['movieId','userId'])

In [28]:
single.show()

+-------+------+
|movieId|userId|
+-------+------+
|     11|    17|
|     20|    17|
|     22|    17|
|     29|    17|
|     33|    17|
|     46|    17|
|     58|    17|
|     68|    17|
|     69|    17|
|     73|    17|
|     78|    17|
+-------+------+



In [29]:
recommendations = model.transform(single)

In [31]:
recommendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     22|    17|  2.6020937|
|     73|    17|  1.2157385|
|     78|    17| 0.28234255|
|     11|    17|0.025712699|
|     20|    17| -0.8252182|
|     29|    17|  -1.881937|
|     58|    17| -2.1723635|
|     69|    17|  -2.466783|
|     68|    17| -3.2372751|
|     46|    17| -3.4741127|
|     33|    17| -6.3127995|
+-------+------+-----------+

